In [5]:
from collections import OrderedDict

import torch
import torch.nn as nn

In [4]:
d_model = 768
n_head = 12
model = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_head)

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
)

In [ ]:
class LayerNorm(nn.LayerNorm):
    
    def forward(self, x:torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)

In [ ]:
class QuickGELU(nn.Module):
    def forward(self, x: torch.Tensor):
        # 这里是利用了sigmoid对GELU的计算进行了简化？
        return x * torch.sigmoid(1.702 * x)

In [ ]:
class ResidualAttentionBlock(nn.Module):
    def __init__(self, d_model, n_head, attn_mask=None):
        super(ResidualAttentionBlock, self).__init__()
        
        self.attn = nn.MultiheadAttention(d_model, n_head)
        self.ln_1 = LayerNorm(d_model)
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc", nn.Linear(d_model, d_model * 4)),
            ("gelu", QuickGELU()),
            ("c_proj", nn.Linear(d_model * 4, d_model))
        ]))
        self.ln_2 = LayerNorm(d_model)
        self.attn_mask = attn_mask
        
    def attention(self, x: torch.Tensor):
        